In [19]:
import statsmodels.formula.api as smf
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import scipy.stats as stats
import pylab
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import export_graphviz
import pydot


data_path = "airline_2m.csv"

#CSV Can be directly downloaded as a tar file here: 

#https://dax-cdn.cdn.appdomain.cloud/dax-airline/1.0.1/airline_2m.tar.gz?_ga=2.241493383.460169643.1645306071-17791737.1643504108

# Credit to IBM's Airline Dataset for the data and encoding code.


data = pd.read_csv(data_path, encoding = "ISO-8859-1",
                 dtype={'Div1Airport': str, 'Div1TailNum': str, 'Div2Airport': str, 'Div2TailNum': str})
#Code Provided By IBMs Airline Reporting Carrier on-time performance set. 
#This helps python better interpret the data frame because it is not encoded in UTF-8 
#An alternative to this could be to actually change the file to be encoded in UTF-8
#but this was difficult to do with how big the file was.


## One-Hot Encoding Splitting Label and Features

The chosen features are destination, year, time in the sky, month, airline and originating airport. The label trying to be predicted is arrival delay.  

In [20]:
sample = data.sample(1000)

sample = sample[['Origin','Dest','ArrDelay', 'Year','AirTime','Month','Reporting_Airline']]
sample['ArrDelay'] = sample['ArrDelay'].fillna(0)
sample['AirTime'] = sample['AirTime'].fillna(0)
print(sample.columns)
features = pd.get_dummies(sample)

print(features.head())

# Labels are the values we want to predict
labels = np.array(features['ArrDelay'])
# Remove the labels from the features
# axis 1 refers to the columns
features = features.drop('ArrDelay', axis = 1)
# Saving feature names for later use
feature_list = list(features.columns)
# Convert to numpy array
features = np.array(features)




Index(['Origin', 'Dest', 'ArrDelay', 'Year', 'AirTime', 'Month',
       'Reporting_Airline'],
      dtype='object')
         ArrDelay  Year  AirTime  Month  Origin_ABQ  Origin_AEX  Origin_AGS  \
811960       24.0  2017     54.0      3           0           0           0   
1523576      -4.0  1994      0.0      8           0           0           0   
1296041     -13.0  2004     95.0      5           0           0           0   
431851        5.0  2009     68.0      4           0           0           0   
513172       21.0  2000    116.0      5           0           0           0   

         Origin_ALB  Origin_AMA  Origin_ANC  ...  Reporting_Airline_PA (1)  \
811960            0           0           0  ...                         0   
1523576           0           0           0  ...                         0   
1296041           0           0           0  ...                         0   
431851            0           0           0  ...                         0   
513172            0

# Splitting the Data Between Test and Train

In [21]:
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 420)

print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (750, 327)
Training Labels Shape: (750,)
Testing Features Shape: (250, 327)
Testing Labels Shape: (250,)


# Create Base Line Predictions

The baseline prediction is an array of 0s representing a baseline prediction of no delay. This was chosen as 0 is the expected value of delay. 

In [31]:
# The baseline predictions is 0
baseline_preds = np.zeros(len(test_features))
print(baseline_preds.shape)
# Baseline errors, and display average baseline error
baseline_errors = abs(baseline_preds - test_labels)
print(baseline_errors)
print('Average baseline error: ', round(np.mean(baseline_errors), 2))

(250,)
[  6.  47.  15.   8.   7.   2.  10.   4.   8.   2. 117.  31.   9.  14.
   5.  10.  14.  18.   8.  20.   0.  25.  15.  24.   4.   7.  10.   3.
   0.  27.   3.   1.  30.  12.   4.  39.   0.  32.   7.   3. 230.   1.
  13.   6.  13.  53.   8.   4.  40.  30.   3.   0.   2.  13.   5.  10.
   9.  20.  12.   6.  27.   5.  45.  11. 103.  17.  21.   5.  12.   2.
   3.   1.  41.   4.  11.  15.   5.   1.   6.   1.   7.  13.  13.  47.
   8.   2.  10.   4.   2.   3.  10.  10.   3.   4.  21.  14.  47.   8.
  11.  74.  12.  56.  38.  10.  12.   8.  12.  28.  21.  10.   9.  19.
  23.   9.  10.  24.  14.   3.  89.   4.   0.   7.   3.   2.   0.  18.
  15.   7.   3.   7.   9.   8.   0.   0.  14.  29.   3.   5.   6.   1.
  25.  12.  14.  16.   5.   5.   2.   0.   4.  11.  31.   7.  27.  18.
  14.   6.  12.   7.  99.   2.   3.  12.   1.  17.   5.   7.  15.   9.
   3.  21.   3.   7.  64.   0.   5.  15.   3.  15.   0.  71.   6.   7.
   4.   4.  15.  25.   0.   3.   1.  11.   2.   4.  15.   4.   2.   4.

### Train the model

In [28]:
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 100, random_state = 420)
# Train the model on training data
rf.fit(train_features, train_labels);
# Use the forest's predict method on the test data


In [41]:
rf_new = RandomForestRegressor(n_estimators = 100, criterion = 'mse', max_depth = None, 
                               min_samples_split = 2, min_samples_leaf = 1)
rf_new.fit(train_features, train_labels);

### Test the model
Notice that the average Baseline error was lower average estimate error. This indicates that the model has issues and cannot reliably predict the label. The label chosen was Arrival delay.

In [42]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 18.95 degrees.


### Generates list of the Features by importance
Notice that flights that arrive in Minneapolis Saint Paul is the most important factor, perhaps this is because of the weather in Minnesota.


In [25]:

# Get numerical feature importances
importances = list(rf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 4)) 
                       for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: Year                 Importance: 0.0796
Variable: AirTime              Importance: 0.071
Variable: Month                Importance: 0.0683
Variable: Dest_RSW             Importance: 0.0428
Variable: Dest_LAS             Importance: 0.0422
Variable: Origin_ORF           Importance: 0.0414
Variable: Reporting_Airline_UA Importance: 0.0297
Variable: Origin_SRQ           Importance: 0.0278
Variable: Reporting_Airline_DL Importance: 0.0254
Variable: Origin_PVD           Importance: 0.0251
Variable: Dest_SLC             Importance: 0.0249
Variable: Dest_LGA             Importance: 0.0213
Variable: Dest_CMI             Importance: 0.0206
Variable: Origin_SFO           Importance: 0.0202
Variable: Dest_MOB             Importance: 0.019
Variable: Origin_ALB           Importance: 0.018
Variable: Origin_PIT           Importance: 0.0178
Variable: Origin_EWR           Importance: 0.0175
Variable: Origin_MDW           Importance: 0.0173
Variable: Dest_XNA             Importance: 0.0132
Var

In [26]:
# New random forest with only the two most important variables
rf_most_important = RandomForestRegressor(n_estimators= 1000, random_state=42)
# Extract the two most important features
important_indices = [feature_list.index('Distance'), feature_list.index('AirTime')]
train_important = train_features[:, important_indices]
test_important = test_features[:, important_indices]
# Train the random forest
rf_most_important.fit(train_important, train_labels)
# Make predictions and determine the error
predictions = rf_most_important.predict(test_important)
errors = abs(predictions - test_labels)
# Display the performance metrics
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')


ValueError: 'Distance' is not in list